## Visual Studio Code (VSCode) - Part 2 of 2

**Estimated Time To Complete This Notebook: 1 hour.**

We will continue exploring Visual Studio Code together in this notebook.

We previously created an ASP.NET Web API backend for a <b>Todo</b> application.

In this notebook, we will create the frontend for the <b>Todo</b> application, adding the project below to the same Solution as the ASP.NET Web API backend:

- `Todo.Blazor` which is a Blazor project with non-interactive Server Side Rendering (SSR), used as the application's front end with a user interface for managing `TodoItem`s.

---

### Add the `Todo.Blazor` Project to the `Todo` Solution via `Solution Explorer`

Now that we have created and debugged the Todo application's backend, let's add a Blazor web front end.

- Right-click the `Todo` solution node in `Solution Explorer` and choose `New Project`.
- In the `Command Palette` choose:
  - Create a new .NET project: `Blazor Web App`
  - Name the new project: `Todo.Blazor`
  - Select location for the new project: `Default Directory`
  - Create project or view options: `Show all template options`
  - Select an option to configure: `Interactive render mode`
  - Configure option Interactive render mode: `None`
  - Select an option to configure: `Include sample pages`
  - Configure option Include sample pages: `False`
  - Select an option to configure: `Create project`
- This will add the `Todo.Blazor` project to the Solution (notice the new `Todo.Blazor` project node under the Solution node `Todo` in `Solution Explorer`).

Next, let's add the required NuGet packages to the `Todo.Blazor` project:

- We need the `AutoMapper` NuGet Package:
  - Right-click the `Todo.Blazor` project node in `Solution Explorer` and choose `Add NuGet Package...`
  - In the `Command Palette`, enter the search term `AutoMapper` and hit `<Enter>`
  - Select `AutoMapper`
  - Choose `13.0.1` (which is currently the latest **compatible** version)

The `Todo.Blazor` project needs to reference the `Todo.Dto` project:

- Right-click the `Todo.Blazor` project in `Solution Explorer`, choose `Add Project Reference`, and select `Todo.Dto` in the `Command Palette`.

Let's create a `TodoItem` model class that we can use in the front end:

- Right-click the `Todo.Blazor` project in `Solution Explorer`, choose `New Folder`, and name the folder `Models`.
- Right-click the `Models` folder, choose `New File` and the file type to `Class`, and name the file `TodoItem`.
- Open the `TodoItem.cs` file and replace its contents with the code below:

```csharp
using System.ComponentModel.DataAnnotations;

namespace Todo.Blazor.Models;

public class TodoItem
{
    public int Id { get; set; }
    [Required(ErrorMessage = "The TaskName field is required."), StringLength(25)]
    public string TaskName { get; set; } = null!;
    [Required(ErrorMessage = "The Notes field is required."), StringLength(500)]
    public string Notes { get; set; } = null!;
    public bool Done { get; set; }
}
```

We need to add a `TodoItemProfile` class for `AutoMapper`, so `Todo.Blazor` can map:

- Outgoing `TodoItem` models to `TodoItemRequestDto`s.
- Incoming `TodoItemResponseDto`s to `TodoItem` models.

To do this:

- Right-click the `Todo.Blazor` project in `Solution Explorer`, choose `New Folder`, and name the folder `Profiles`.
- Right-click the `Profiles` folder, choose `New File` and the file type to `Class`, and name the file `TodoItemProfile`.
- Open the `TodoItemProfile.cs` file and replace its contents with the code below:

```csharp
using AutoMapper;
using Todo.Dto;
using Todo.Blazor.Models;

namespace Todo.Blazor.Profiles;

public class TodoItemProfile : Profile
{
    public TodoItemProfile()
    {
        // Map Model to DTO
        // Note that "src.Id" isn't mapped to any "Id" property in the DTO
        // Note that "dest.Name" gets its value from "src.TaskName"
        CreateMap<TodoItem, TodoItemRequestDto>()
        .ForMember(dest => dest.Name, opt => opt.MapFrom(src => src.TaskName))
        .ForMember(dest => dest.Notes, opt => opt.MapFrom(src => src.Notes))
        .ForMember(dest => dest.Done, opt => opt.MapFrom(src => src.Done));

        // Map DTO to Model
        // Note that "dest.TaskName" gets its value from "src.Name"
        CreateMap<TodoItemResponseDto, TodoItem>()
        .ForMember(dest => dest.Id, opt => opt.MapFrom(src => src.Id))
        .ForMember(dest => dest.TaskName, opt => opt.MapFrom(src => src.Name))
        .ForMember(dest => dest.Notes, opt => opt.MapFrom(src => src.Notes))
        .ForMember(dest => dest.Done, opt => opt.MapFrom(src => src.Done));

        //CreateMap<TodoItem, TodoItemDto>().ReverseMap();
    }
}
```

Next, let's create a Rest service we can use to communicate with the ASP.NET Core Web API backend.

- Right-click the `Todo.Blazor` project in `Solution Explorer`, choose `New Folder`, and name the folder `Services`.
- Right-click the `Services` folder, choose `New File` and the file type to `Interface`, and name the file `IRestService`.
- Open the `IRestService.cs` file and replace its contents with the code below:

```csharp
using Todo.Blazor.Models;

namespace Todo.Blazor.Services;

public interface IRestService
{
    Task<List<TodoItem>> GetTodoItemsAsync();
    Task<TodoItem> GetTodoItemAsync(int id);
    Task AddTodoItemAsync(TodoItem todoItem);
    Task UpdateTodoItemAsync(TodoItem todoItem);
    Task DeleteTodoItemAsync(int id);
}
```

Let's create a corresponsing `RestService` class:

- Right-click the `Todo.Blazor` project in `Solution Explorer`, choose `New Folder`, and name the folder `Services`.
- Right-click the `Services` folder, choose `New File` and the file type to `Class`, and name the file `RestService`.
- Open the `RestService.cs` file and replace its contents with the code below:

```csharp
using AutoMapper;
using System.Diagnostics;
using System.Text;
using System.Text.Json;
using Todo.Dto;
using Todo.Blazor.Models;

namespace Todo.Blazor.Services;

public class RestService : IRestService
{
    private readonly HttpClient _httpClient;
    private JsonSerializerOptions _serializerOptions;
    private readonly IMapper _mapper;

    public List<TodoItem>? Items { get; private set; }

    public RestService(HttpClient httpClient, IMapper mapper)
    {
        _mapper = mapper;
        _httpClient = httpClient;
        _serializerOptions = new JsonSerializerOptions { PropertyNamingPolicy = JsonNamingPolicy.CamelCase, WriteIndented = true };
    }

    public async Task<List<TodoItem>> GetTodoItemsAsync()
    {
        // The single line below is enough for the entire method body, but then without detailed error checking
        //return _mapper.Map<List<TodoItem>> (await _httpClient.GetFromJsonAsync<List<TodoItemResponseDto>>("/api/todoitems", _serializerOptions) ?? []);
        
        Items = new List<TodoItem>();
        try
        {
            HttpResponseMessage response = await _httpClient.GetAsync("/api/todoitems");
            if (response.IsSuccessStatusCode)
            {
                string content = await response.Content.ReadAsStringAsync();
                Items = _mapper.Map<List<TodoItem>>(JsonSerializer.Deserialize<List<TodoItemResponseDto>>(content, _serializerOptions));
                // The two lines above can be replaced with the single line below
                //Items = _mapper.Map<List<TodoItem>>(await response.Content.ReadFromJsonAsync<List<TodoItemResponseDto>>(_serializerOptions));
            }
        }
        catch (Exception ex)
        {
            Debug.WriteLine(@"\tERROR {0}", ex.Message);
        }
        return Items;
    }

    // This method shows a one-line implementation, but lacks detailed error checking
    public async Task<TodoItem> GetTodoItemAsync(int id)
        => _mapper.Map<TodoItem> (await _httpClient.GetFromJsonAsync<TodoItemResponseDto>($"/api/todoitems/{id}", _serializerOptions) ?? throw new Exception("Could not find TodoItem"));

    public async Task AddTodoItemAsync(TodoItem todoItem)
    {
        // The single line below is enough for the entire method body, but then without detailed error checking
        //await _httpClient.PostAsJsonAsync("/api/todoitems", _mapper.Map<TodoItemRequestDto>(todoItem), _serializerOptions);

        try
        {
            string json = JsonSerializer.Serialize<TodoItemRequestDto>(_mapper.Map<TodoItemRequestDto>(todoItem), _serializerOptions);
            StringContent content = new StringContent(json, Encoding.UTF8, "application/json");
            HttpResponseMessage response = null!;
            response = await _httpClient.PostAsync($"/api/todoitems", content);
            if (response.IsSuccessStatusCode)
                Debug.WriteLine(@"\tTodoItem successfully created.");
        }
        catch (Exception ex)
        {
            Debug.WriteLine(@"\tERROR {0}", ex.Message);
        }
    }

    public async Task UpdateTodoItemAsync(TodoItem todoItem)
    {
        // The single line below is enough for the entire method body, but then without detailed error checking
        //await _httpClient.PutAsJsonAsync($"/api/todoitems/{todoItem.Id}", _mapper.Map<TodoItemRequestDto>(todoItem), _serializerOptions);

        try
        {
            string json = JsonSerializer.Serialize<TodoItemRequestDto>(_mapper.Map<TodoItemRequestDto>(todoItem), _serializerOptions);
            StringContent content = new StringContent(json, Encoding.UTF8, "application/json");
            HttpResponseMessage response = null!;
            response = await _httpClient.PutAsync($"/api/todoitems/{todoItem.Id}", content);
            if (response.IsSuccessStatusCode)
                Debug.WriteLine(@"\tTodoItem successfully updated.");
        }
        catch (Exception ex)
        {
            Debug.WriteLine(@"\tERROR {0}", ex.Message);
        }
    }

    public async Task DeleteTodoItemAsync(int id)
    {
        // The single line below is enough for the entire method body, but then without detailed error checking
        //await _httpClient.DeleteAsync($"/api/todoitems/{id}");

        try
        {
            HttpResponseMessage response = await _httpClient.DeleteAsync($"/api/todoitems/{id}");
            if (response.IsSuccessStatusCode)
                Debug.WriteLine(@"\tTodoItem successfully deleted.");
        }
        catch (Exception ex)
        {
            Debug.WriteLine(@"\tERROR {0}", ex.Message);
        }
    }
}
```

Now we need to add our `RestService` and `AutoMapper` as services to the WebbApplication's Service Container.

- Open the file `Program.cs` under the `Todo.Blazor` project in `Solution Explorer` and replace its contents with the code below:

```csharp
using Todo.Blazor.Components;
using Todo.Blazor.Services;

var builder = WebApplication.CreateBuilder(args);

// Add services to the container.
builder.Services.AddRazorComponents().AddInteractiveServerComponents();

var todoApiUrl = builder.Configuration["TodoApiUrl"] ?? throw new Exception("TodoApiUrl is not set");
builder.Services.AddHttpClient<RestService>(configureClient => configureClient.BaseAddress = new Uri(todoApiUrl));
builder.Services.AddAutoMapper(AppDomain.CurrentDomain.GetAssemblies());

var app = builder.Build();

// Configure the HTTP request pipeline.
if (!app.Environment.IsDevelopment())
{
    app.UseExceptionHandler("/Error", createScopeForErrors: true);
    // The default HSTS value is 30 days. You may want to change this for production scenarios, see https://aka.ms/aspnetcore-hsts.
    app.UseHsts();
}

app.UseHttpsRedirection();

app.UseStaticFiles();
app.UseAntiforgery();

app.MapRazorComponents<App>().AddInteractiveServerRenderMode();

app.Run();
```

We also need to add the `TodoApiUrl` to the `Todo.Blazor` project's `appsettings.json` file, so open it and replce its contents with the code below:

```csharp
{
  "Logging": {
    "LogLevel": {
      "Default": "Information",
      "Microsoft.AspNetCore": "Warning"
    }
  },
  "TodoApiUrl": "http://localhost:5000",
  "AllowedHosts": "*"
}
```

Let's also edit the file `launchsettings.json` in the `Properties` folder, under the `Todo.Blazor` project.

- Open the file `launchsettings.json` in the `Properties` folder, under the `Todo.Blazor` project, in `Solution Explorer`, and replace its contents with the code below:

```json
{
  "$schema": "https://json.schemastore.org/launchsettings.json",
  "profiles": {
    "http": {
      "commandName": "Project",
      "dotnetRunMessages": true,
      "launchBrowser": true,
      "applicationUrl": "http://localhost:5010",
      "environmentVariables": {
        "ASPNETCORE_ENVIRONMENT": "Development"
      }
    },
    "https": {
      "commandName": "Project",
      "dotnetRunMessages": true,
      "launchBrowser": true,
      "applicationUrl": "https://localhost:5011;http://localhost:5010",
      "environmentVariables": {
        "ASPNETCORE_ENVIRONMENT": "Development"
      }
    }
  }
}
```

**Note**

We are setting `launchBrowser: true` so that a browser in launched when debugging the project.

We are also changing the port to `5010` for the `http` scheme, and to `5011` for the `https` scheme.

When debugging the project, we can access the web frontend via the URL: [http://localhost:5010](http://localhost:6000).

Next, let's add support for [Bootstrap](https://getbootstrap.com).

- Open the file `App.razor` in the `Components` folder under the `Todo.Blazor` project, and replace its contents with the code below:

```html
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <base href="/" />
    <link rel="stylesheet" href="app.css" />
    <link rel="stylesheet" href="Todo.Blazor.styles.css" />
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-QWTKZyjpPEjISv5WaRU9OFeRpok6YctnYmDr5pNlyT2bRjXh0JMhjY6hW+ALEwIH" crossorigin="anonymous">
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap-icons@1.11.3/font/bootstrap-icons.min.css">
    <HeadOutlet />
</head>

<body>
    <Routes />
    <script src="_framework/blazor.web.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/js/bootstrap.bundle.min.js" integrity="sha384-YvpcrYf0tY3lHB60NNkmXc5s9fDVZLESaAA55NDzOxhy9GkcIdslK1eN7N6jIeHz" crossorigin="anonymous"></script>

</body>

</html>
```

Let's also add the namespaces for our Models and Servies to the file `_Imports.razor`, so we don't have to include them in all our components:

- Open the file `_Imports.razor` in the `Components` folder, and replace its contents with the code below:

```csharp
@using System.Net.Http
@using System.Net.Http.Json
@using Microsoft.AspNetCore.Components.Forms
@using Microsoft.AspNetCore.Components.Routing
@using Microsoft.AspNetCore.Components.Web
@using static Microsoft.AspNetCore.Components.Web.RenderMode
@using Microsoft.AspNetCore.Components.Web.Virtualization
@using Microsoft.JSInterop
@using Todo.Blazor
@using Todo.Blazor.Components
@using Todo.Blazor.Models
@using Todo.Blazor.Services
```

We need a navigation menu, so let's add a component for this:

- Right-click the `Layout` folder under the `Components` folder in `Solution Explorer`, and choose `New file`.
- In the `Command Palette`, choose the file type to `Razor Component`, and name the component `NavMenu`.
- Open the file `NavMenu.razor` and replace its contents with the code below:

```html
<nav class="navbar bg-body-tertiary" data-bs-theme="dark">
  <div class="container">
    <span class="navbar-brand mb-0 h1">Todo</span>
  </div>
</nav>
```

Now, let's add our new navigation menu to the file `MainLayout.razor`, also in the `Layout` folder:

- Open the file `MainLayout.razor` and replace its contents with the code below:

```html
@inherits LayoutComponentBase

<NavMenu/>
<div class="container">
@Body
</div>
```

Next, we need a view to list, edit and delete our todo items. Let's start with a component for editing a todo item:

- Right-click the `Pages` folder under the `Components` folder in `Solution Explorer`, and choose `New file`.
- In the `Command Palette`, choose the file type to `Razor Component`, and name the component `EditTodoItem`.
- Open the file `EditTodoItem.razor` and replace its contents with the code below:

```csharp
@page "/edittodoitem"
@page "/edittodoitem/{id:int}"

@rendermode InteractiveServer

@inject NavigationManager NavigationManager
@inject RestService RestService

<PageTitle>@title</PageTitle>

<h3>@title</h3>

@if (todoItem is null)
{
    <p><em>Loading...</em></p>
}
else
{
    <div class="row">
        <div class="col-md-4">
            <EditForm Model="@todoItem" FormName="editTodoItem" OnValidSubmit="HandleSubmitAsync">
                <DataAnnotationsValidator />
                <ValidationSummary />
                <div class="mb-3">
                    <label for="id" class="form-label">Id</label>
                    <InputNumber id="id" @bind-Value="todoItem.Id" class="form-control" readonly />
                </div>
                <div class="mb-3">
                    <label for="taskname" class="form-label">Name</label>
                    <InputText id="taskname" @bind-Value="todoItem.TaskName" class="form-control" />
                    <ValidationMessage For="() => todoItem.TaskName"/>
                </div>
                <div class="mb-3">
                    <label for="notes" class="form-label">Notes</label>
                    <InputText id="notes" @bind-Value="todoItem.Notes" class="form-control" />
                    <ValidationMessage For="() => todoItem.Notes"/>
                </div>
                <div class="mb-3">
                    <label for="done" class="form-label">Done</label>
                    <InputCheckbox id="done" @bind-Value="todoItem.Done" class="form-check-input" />
                </div>
                <button type="submit" class="btn btn-primary">Save</button>
                <a role="button" class="btn btn-secondary" href="/">Cancel</a>
            </EditForm>
        </div>
    </div>
}

@code {
    [Parameter]
    public int? Id {get; set;}

    [SupplyParameterFromForm]
    private TodoItem? todoItem {get; set;}

    private string title = string.Empty;

    protected async override Task OnParametersSetAsync()
    {
        // Form submitted
        if (todoItem is not null)
        {
            return;
        }

        // Form presented
        if (Id is not null)
        {
            todoItem = await RestService.GetTodoItemAsync(Id.Value);
            title = $"Edit {todoItem.TaskName}";
        }
        else
        {
            todoItem = new()
            {
                Id = 0,
                TaskName = string.Empty,
                Notes = string.Empty,
                Done = false
            };

            title = "New TodoItem";
        }
    }

    private async Task HandleSubmitAsync()
    {
        ArgumentNullException.ThrowIfNull(todoItem);

        if (Id is null)
        {
            await RestService.AddTodoItemAsync(todoItem);
        }
        else
        {
            todoItem.Id = Id.Value;
            await RestService.UpdateTodoItemAsync(todoItem);
        }
        
        NavigationManager.NavigateTo("/");
    }
}
```

Now, let's add a component for confirming the deletion of a todo item:

- Right-click the `Pages` folder under the `Components` folder in `Solution Explorer`, and choose `New file`.
- In the `Command Palette`, choose the file type to `Razor Component`, and name the component `DeleteTodoItem`.
- Open the file `DeleteTodoItem.razor` and replace its contents with the code below:

```csharp
@inject RestService RestService
@inject NavigationManager NavigationManager
@rendermode InteractiveServer

<div class="modal fade" id="@GetModalId(TodoItem)" tabindex="-1" aria-labelledby="exampleModalLabel" aria-hidden="true">
  <div class="modal-dialog">
    <div class="modal-content">
      <div class="modal-header">
        <h1 class="modal-title fs-5" id="exampleModalLabel">@title</h1>
        <button type="button" class="btn-close" data-bs-dismiss="modal" aria-label="Close"></button>
      </div>
      <div class="modal-footer">
        <button type="button" class="btn btn-secondary" data-bs-dismiss="modal">Cancel</button>
        <button type="button" class="btn btn-primary" data-bs-dismiss="modal" @onclick="@ConfirmAsync">Delete</button>
      </div>
    </div>
  </div>
</div>

@code {
    [Parameter]
    public TodoItem? TodoItem {get; set;}

    private string title = string.Empty;

    protected override void OnParametersSet()
    {
        title = $"Delete {TodoItem?.TaskName}?";
    }

    public static string GetModalId(TodoItem? todoItem)
    {
        ArgumentNullException.ThrowIfNull(todoItem);
        return $"deleteModal-{todoItem.Id}";
    }

    private async Task ConfirmAsync()
    {
        await RestService.DeleteTodoItemAsync(TodoItem!.Id);
        NavigationManager.Refresh();
    }
}
```

Finally, let's list our todo items in the file `Home.razor`.

- Open the file `Home.razor` in the `Pages` folder and replace its contents with the code below:

```csharp
@page "/"
@inject RestService RestService

<PageTitle>Todo List</PageTitle>

<div class="mt-2">
    <a class="btn btn-primary" role="button" href="/edittodoitem">New TodoItem</a>
</div>

@if (todoItems is null)
{
    <p><em>Loading...</em></p>
}
else
{
    <table class="table table-striped table-bordered table-hover mt-3">
        <thead class="table-dark">
            <tr>
                <th class="text-end">Id</th>
                <th>Name</th>
                <th>Done</th>
                <th></th>
            </tr>
        </thead>
        <tbody>
            @foreach (var item in todoItems)
            {
                <tr>
                    <td class="text-end">@item.Id</td>
                    <td>@item.TaskName</td>
                    <td>@item.Done</td>
                    <td>
                        <div class="d-flex">
                            <a class="btn btn-primary me-2" role="button" href="@TodoItemUrl(item.Id)">
                                <i class="bi bi-pencil"></i>
                            </a>
                            <button class="btn btn-danger" data-bs-toggle="modal" data-bs-target="@GetDeleteModalId(item)">
                                <i class="bi bi-x-lg"></i>
                            </button>
                        </div>
                        <DeleteTodoItem TodoItem="@item" />
                    </td>
                </tr>
            }
        </tbody>
    </table>
}

@code
{
    private List<TodoItem>? todoItems;

    protected async override Task OnInitializedAsync()
    {
        todoItems = await RestService.GetTodoItemsAsync();
    }

    private static string TodoItemUrl(int id) => $"/edittodoitem/{id}";

    private string GetDeleteModalId(TodoItem todoItem)
    {
        return $"#{DeleteTodoItem.GetModalId(todoItem)}";
    }
}
```

---

### Debug the Application

Let's debug the entire application:

- Right-click the `Todo.Api` project in `Solution Explorer` and choose `Debug -> Start New Instance`.
- Right-click the `Todo.Blazor` project in `Solution Explorer` and choose `Debug -> Start New Instance`.

<style>
    .container {
        width: 98%;
        margin-left: 0; /* Push the container to the left */
        margin-right: auto; 
    }
    .text-image {
        margin-bottom: 35px; /* Space between sections */
        overflow: hidden; /* Ensure image stays within the container */
    }
    .text {
        text-align: justify; /* Justify the text for better readability */
    }
    .image {
        float: right; /* Float the image to the right */
        margin-left: 25px; /* Space between image and text */
        margin-bottom: 10px; /* Space between image and text */
        max-width: 50%; /* Limit image size */
        height: auto; /* Maintain aspect ratio */
    }
</style>

<div class="container">
    <div class="text-image">
        <img class="image" src="../images/todoitems1.png">
        <div class="text">
            <p>
                Visit <a href="http://localhost:5010">http://localhost:5010</a> which should take you to the Home page, showing a list of todo items.
            </p>
            <p>
                Click the <b>New TodoItem</b> button which should take you to the Edit TodoItem page.
            </p>
        </div>
    </div>
</div>

<div class="container">
    <div class="text-image">
        <img class="image" src="../images/todoitems2.png">
        <div class="text">
            <p>
                On the Edit TodoItem page, fill in the form:
                <ul>
                    <li>TaskName: <b>Yakety Yak</b></li>
                    <li>Notes: <b>Take out the papers and the trash.</b></li>
                    <li>Done: <b>Unchecked</b></li>
                </ul>
            </p>
            <p>
                Click the <b>Save</b> button, which should take you back to the Home page.
            </p>
        </div>
    </div>
</div>

<div class="container">
    <div class="text-image">
        <img class="image" src="../images/todoitems3.png">
        <div class="text">
            <p>
                On the Home page, the new TodoItem should be displayed in the list.
            </p>
            <p>
                Click the <b>Edit</b> (pencil) icon for the new TodoItem, which should take you to the Edit TodoItem page.
            </p>
        </div>
    </div>
</div>

<div class="container">
    <div class="text-image">
        <img class="image" src="../images/todoitems4.png">
        <div class="text">
            <p>
                On the Edit TodoItem page, check the <b>Done</b> box, and then click the <b>Save</b> button, which should take you back to the Home page.
            </p>
        </div>
    </div>
</div>

<div class="container">
    <div class="text-image">
        <img class="image" src="../images/todoitems5.png">
        <div class="text">
            <p>
                On the Home page, the TodoItem's new <b>Done</b> state should be shown.
            </p>
            <p>
                Click the Delete <b>X</b> icon for the TodoItem, which should pop up a confirmation page.
            </p>
        </div>
    </div>
</div>

<div class="container">
    <div class="text-image">
        <img class="image" src="../images/todoitems6.png">
        <div class="text">
            <p>
                On the confirmation page, click the <b>Delete</b> button, which should close the confirmation page.
            </p>
        </div>
    </div>
</div>

<div class="container">
    <div class="text-image">
        <img class="image" src="../images/todoitems1.png">
        <div class="text">
            <p>
                On the Home page, the TodoItem should now be absent in the list.
            </p>
        </div>
    </div>
</div>

Stop debugging the `Todo.Blazor` and `Todo.Api` projects (click the red stop button twice).

---

### Conclusion

This completes the introduction to VSCode, where we have created a complete application with a Web API backend (previous notebook) and a Blazor Web front end (this notebook).

Next, we will look at Unit Testing in VSCode:

- Keep the VSCode instance with the `Todo` solution open (we will continue to use it in the next notebook).
- In the `notebooks` folder, open the file `unittesting.ipynb`.
- When the notebook opens in VSCode, click the text `Select Kernel` (top-right), and choose `Python Environments... => conda (Python 3.10.15) .conda/bin/python`.
- Now you can follow the instructions in the notebook.